
# 第5章: 係り受け解析
日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルがai.ja.zipに収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

## 記事：https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD

## 準備0：mecab,cabochaのインストール

In [1]:
#ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## 準備1：mecab,cabochaのインストール

In [ ]:
%%bash
# mecabとmecab-python3の依存関係をインストール
apt-get install mecab swig libmecab-dev mecab-ipadic-utf8
# mecab-pythonのインストール
pip install mecab-python3

# crfppダウンロード(cabochaの依存関係)
curl -sL -o CRF++-0.58.tar.gz "https://drive.google.com/uc?export=download&id=0B4y35FiV1wh7QVR6VXJ5dWExSTQ"
tar -zxf CRF++-0.58.tar.gz
# crfppインストール
cd CRF++-0.58
./configure && make && make install && ldconfig
cd ..

# cabochaダウンロード
url="https://drive.google.com/uc?export=download&id=0B4y35FiV1wh7SDd1Q1dUQkZQaUU"
curl -sc /tmp/cookie ${url} >/dev/null
code="$(awk '/_warning_/ {print $NF}' /tmp/cookie)"
curl -sLb /tmp/cookie ${url}"&confirm=${code}" -o cabocha-0.69.tar.bz2
tar -jxf cabocha-0.69.tar.bz2
# cabochaインストール
cd cabocha-0.69
./configure -with-charset=utf-8 && make && make check && make install && ldconfig
# cabocha-pythonのインストール
pip install python/
cd ..




Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-jumandic mecab-jumandic-utf8 mecab-utils
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-jumandic
  mecab-jumandic-utf8 mecab-utils swig swig3.0
0 upgraded, 10 newly installed, 0 to remove and 35 not upgraded.
Need to get 30.1 MB of archives.
After this operation, 282 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab-dev amd64 0.996-5 [308 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/u

./configure: line 7077: /usr/bin/file: No such file or directory
In file included from tagger.h:14:0,
                 from feature.cpp:12:
param.h:34:13: warning: 'Target CRFPP::{anonymous}::lexical_cast(Source) [with Target = std::__cxx11::basic_string<char>; Source = std::__cxx11::basic_string<char>]' defined but not used [-Wunused-function]
 std::string lexical_cast<std::string, std::string>(std::string arg) {
             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ar: `u' modifier ignored since `D' is the default (see `U')
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

./configure: line 7604: /usr/bin/file: No such file or directory
In file included from dep_learner.cpp:17:0:
param.h:30:13: warning: 'Target {anonymous}::lexical_cast(Source) [with Target = std::__cxx11::basic_string<char>; Source = std::__cxx11::basic_string<char>]' defined but not used [-Wunused-function]
 std::string lexical_cast<std::string, std::strin

In [ ]:
#colab で mecab をインストール際に、mecabrc が
#/usr/local/etc/mecabrc
#ではなく
#/etc/mecabrc
#に配置されることが原因らしい。
#というわけで、シンボリックリンクを貼る

!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#バージョン確認

%%bash
mecab -v
pip show mecab-python3 | grep -e Name -e Version && echo
cabocha -v && echo
pip show cabocha-python | grep -e Name -e Version 


mecab of 0.996

Name: mecab-python3
Version: 1.0.1

cabocha of 0.69

Name: cabocha-python
Version: 0.69


In [ ]:
#動作確認

import MeCab
tagger = MeCab.Tagger()
print(tagger.parse("隣の客はよく柿食う客だ"))

隣	名詞,一般,*,*,*,*,隣,トナリ,トナリ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
客	名詞,一般,*,*,*,*,客,キャク,キャク
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
よく	副詞,一般,*,*,*,*,よく,ヨク,ヨク
柿	名詞,一般,*,*,*,*,柿,カキ,カキ
食う	動詞,自立,*,*,五段・ワ行促音便,基本形,食う,クウ,クウ
客	名詞,一般,*,*,*,*,客,キャク,キャク
だ	助動詞,*,*,*,特殊・ダ,基本形,だ,ダ,ダ
EOS



In [ ]:
import CaboCha
cp = CaboCha.Parser()
print(cp.parseToString("隣の客はよく柿食う客だ"))

隣の-D        
  客は-------D
    よく---D |
        柿-D |
        食う-D
          客だ
EOS



## 準備２：ai.ja.txt.parsedの作成

In [ ]:
import CaboCha
cp = CaboCha.Parser()

In [ ]:
with open('drive/My Drive/NLP100knock/data/chap5/ai.ja/ai.ja.txt') as indata:
  line=indata.read()

with open('drive/My Drive/NLP100knock/data/chap5/ai.ja/ai.ja.txt.persed.txt', mode='w') as outdata:
  outdata.write(cp.parse(line).toString(CaboCha.FORMAT_LATTICE))

##40.係り受け解析結果の読み込み（形態素）

形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

- クラスの使い方：https://uxmilk.jp/41600
- かぼちゃの使い方：https://qiita.com/ayuchiy/items/c3f314889154c4efa71e

In [ ]:
import re
import MeCab
t = MeCab.Tagger('-Ochasen')
import CaboCha
import pprint

In [ ]:
class Morph:
  
  def __init__(self, surface, base, pos, pos1):
    self.surface=surface
    self.base=base
    self.pos=pos
    self.pos1=pos1

def get_morphs(persed_file):
  with open(persed_file) as indata:
      data = indata.read()
      data = data.split('\n')
      
      morphs = []
      for line in data:
        line = re.split(r'[\t,]',line)
        if line[0] not in ('','EOS') and line[0].startswith("*") is False:
          morphs_temp={'surface':line[0], 'base':line[7], 'pos':line[1], 'pos1':line[2]}
          morphs.append(morphs_temp)
      return morphs

if __name__ == "__main__":
    persed_file='drive/My Drive/NLP100knock/data/chap5/ai.ja/ai.ja.txt.persed'
    morphs = get_morphs(persed_file)
    pprint.pprint(morphs[:10])

[{'base': '人工', 'pos': '名詞', 'pos1': '一般', 'surface': '人工'},
 {'base': '知能', 'pos': '名詞', 'pos1': '一般', 'surface': '知能'},
 {'base': '人工', 'pos': '名詞', 'pos1': '一般', 'surface': '人工'},
 {'base': '知能', 'pos': '名詞', 'pos1': '一般', 'surface': '知能'},
 {'base': '（', 'pos': '記号', 'pos1': '括弧開', 'surface': '（'},
 {'base': 'じん', 'pos': '名詞', 'pos1': '一般', 'surface': 'じん'},
 {'base': 'こうち', 'pos': '名詞', 'pos1': '一般', 'surface': 'こうち'},
 {'base': 'のう', 'pos': '助詞', 'pos1': '終助詞', 'surface': 'のう'},
 {'base': '、', 'pos': '記号', 'pos1': '読点', 'surface': '、'},
 {'base': '、', 'pos': '記号', 'pos1': '読点', 'surface': '、'}]


## 41.係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

In [1]:
import re
import numpy as np
import pandas as pd

In [3]:
class Morph:
  
  def __init__(self, surface, base, pos, pos1):
    self.surface=surface
    self.base=base
    self.pos=pos
    self.pos1=pos1

class Chunk:
  def __init__(self, morphs, dst, srcs):
    self.morphs=morphs
    self.dst=dst
    self.srcs=srcs

def get_morphs(persed_file):
  with open(persed_file) as indata:
      data = indata.read()
      data = data.split('\n')
      
      morphs = []
      for line in data:
        line = re.split(r'[\t,]',line)
        if line[0] not in ('','EOS') and line[0].startswith("*") is False:
          morphs_temp={'surface':line[0], 'base':line[7], 'pos':line[1], 'pos1':line[2]}
          morphs.append(morphs_temp)

      return morphs

def get_chunk(persed_file):
  with open(persed_file) as indata:
      data = indata.read()
      data = data.split('\n')

      #文節格納処理と最終的な格納先
      chunk = []
      chunk_temp = []
      count = 0

      #係受け元先番号格納用
      chunk_id=[]
      chunk_to = []

      for i in range(0,len(data)):
        data[i] = re.split(r'[\t,\s]',data[i])
        if data[i][0]=="*" :
          chunk_id.append(data[i][1])
          chunk_to.append(data[i][2].replace('D',''))
          count += 1

        elif data[i][0] not in ('','EOS') and data[i][0]!="*" :
          if count == 1:
            chunk_temp.append(data[i][0])
            
          elif count == 2:
            chunk_temp=''.join(chunk_temp)
            chunk.append(chunk_temp)
            chunk_temp = []
            chunk_temp.append(data[i][0])
    
            count=1
        
        elif data[i][0]=='EOS':
          chunk_temp=''.join(chunk_temp)
          chunk.append(chunk_temp)

      chunk=np.array(chunk)
      chunk_id=np.array(chunk_id)
      chunk_to=np.array(chunk_to)

      #結果をpandasに格納しデータフレームとして表現
      #本当はこれをnumpyでやりたかった
      
      chunk_frame=pd.DataFrame(np.arange(len(chunk)*3).reshape(len(chunk),3),columns=['chunk','chunk_id','chunk_to'])

      chunk_frame['chunk']=chunk
      chunk_frame['chunk_id']=chunk_id
      chunk_frame['chunk_to']=chunk_to

      return chunk_frame

if __name__ == "__main__":
    persed_file='../data/chap5/ai.ja/ai.ja.txt.persed'
    #persed_file='drive/My Drive/NLP100knock/data/chap5/ai.ja/ai.ja.txt.persed'
    morphs = get_morphs(persed_file)
    chunks = get_chunk(persed_file)

In [4]:
chunks.tail()

chunk chunk_id chunk_to
2937         発言し、     2937     2941
2938         伊勢田は     2938     2941
2939         「決着は     2939     2940
2940  つかないでしょうね」と     2940     2941
2941       答えている。     2941       -1

## 過程

In [16]:
persed_file='drive/My Drive/NLP100knock/data/chap5/ai.ja/ai.ja.txt.persed'

In [68]:
  with open(persed_file) as indata:
      data = indata.read()
      data = data.split('\n')

      #print(data)

      #文節格納処理と最終的な格納先
      chunk = []
      chunk_temp = []
      count = 0

      #係受け元先番号格納用
      chunk_id=[]
      chunk_to = []

      for i in range(0,len(data)):
        data[i] = re.split(r'[\t,\s]',data[i])
        if data[i][0]=="*" :
          chunk_id.append(data[i][1])
          chunk_to.append(data[i][2].replace('D',''))
          count += 1
          #print(data[i])

        elif data[i][0] not in ('','EOS') and data[i][0]!="*" :
          if count == 1:
            #print(data[i])
            chunk_temp.append(data[i][0])
            
          elif count == 2:
            #print(chunk_temp)
            chunk.append(chunk_temp)
            chunk_temp = []
            chunk_temp.append(data[i][0])
    
            count=1
        
        elif data[i][0]=='EOS':
          chunk.append(chunk_temp)

  chunk=np.array(chunk)
  chunk_id=np.array(chunk_id)
  chunk_to=np.array(chunk_to)

  #結果をpandasに格納しデータフレームとして表現
  #本当はこれをnumpyでやりたかった

  # chunk_frame=pd.DataFrame(np.arange(len(chunk)*3).reshape(len(chunk),3),columns=['chunk','chunk_id','chunk_to'])

  # chunk_frame['chunk']=chunk
  # chunk_frame['chunk_id']=chunk_id
  # chunk_frame['chunk_to']=chunk_to



In [72]:
chunk[-1]

['答え', 'て', 'いる', '。']

In [69]:
len(chunk)

2942

In [70]:
chunk_id[-1]

'2941'

In [71]:
chunk_temp

['答え', 'て', 'いる', '。']

In [28]:
chunk_frame.values

array([[list(['人工', '知能', '人工', '知能']), '0', '17'],
       [list(['（', 'じん', 'こうち', 'のう', '、', '、']), '1', '17'],
       [list(['AI']), '2', '3'],
       ...,
       [list(['伊勢田', 'は']), '2938', '2941'],
       [list(['「', '決着', 'は']), '2939', '2940'],
       [list(['つか', 'ない', 'でしょ', 'う', 'ね', '」', 'と']), '2940', '2941']],
      dtype=object)

In [5]:
chunk_to

array(['17D', '17D', '3D', ..., '2941D', '2940D', '2941D'], dtype='<U5')

In [6]:
chunk

array([list(['人工', '知能', '人工', '知能']),
       list(['（', 'じん', 'こうち', 'のう', '、', '、']), list(['AI']), ...,
       list(['伊勢田', 'は']), list(['「', '決着', 'は']),
       list(['つか', 'ない', 'でしょ', 'う', 'ね', '」', 'と'])], dtype=object)

In [7]:
chunk_frame=pd.DataFrame(np.arange(len(chunk)*2).reshape(len(chunk),2),columns=['chunk','chunk_to'])

chunk_frame['chunk']=chunk
chunk_frame['chunk_to']=chunk_to

chunk_frame

chunk chunk_to
0                [人工, 知能, 人工, 知能]      17D
1          [（, じん, こうち, のう, 、, 、]      17D
2                            [AI]       3D
3        [〈, エーアイ, 〉, ）, と, は, 、]      17D
4                      [「, 『, 計算]       5D
...                           ...      ...
2936  [つく, の, でしょ, う, か, ？, 」, と]    2937D
2937                   [発言, し, 、]    2941D
2938                     [伊勢田, は]    2941D
2939                   [「, 決着, は]    2940D
2940    [つか, ない, でしょ, う, ね, 」, と]    2941D

[2941 rows x 2 columns]

In [ ]:
len(chunk)

2941

In [ ]:
chunk

[['人工', '知能', '人工', '知能'],
 ['（', 'じん', 'こうち', 'のう', '、', '、'],
 ['AI'],
 ['〈', 'エーアイ', '〉', '）', 'と', 'は', '、'],
 ['「', '『', '計算']]

In [32]:
  with open(persed_file) as indata:
      data = indata.read()
      data = data.split('\n')

      #print(data)

      chunk = []
      chunk_temp = []
      chunk_to = []
      count = 0
      num=[]

      for i in range(0,len(data)):
      #for i in range(0,30):
        data[i] = re.split(r'[\t,\s]',data[i])
        if data[i][0]=="*" :
          #print(data[i][2])
          chunk_to.append(data[i][2])
          count += 1
          num.append(chunk_to)
        elif data[i][0] not in ('','EOS') and data[i][0]!="*" :
          if count == 1:
            pass

          elif count == 2:
          
            chunk_to = []       
            count=1

In [33]:
num[4][0]

'5D'

In [ ]:
chunk[:10]

[['（', 'じん', 'こうち', 'のう', '、', '、'],
 ['AI'],
 ['〈', 'エーアイ', '〉', '）', 'と', 'は', '、'],
 ['「', '『', '計算'],
 ['（', '）', '』', 'という'],
 ['概念', 'と'],
 ['『', 'コンピュータ'],
 ['（', '）', '』', 'という'],
 ['道具', 'を'],
 ['用い', 'て']]

In [ ]:
  with open(persed_file) as indata:
      data = indata.read()
      data = data.split('\n')

      chunk = []
      chunk_temp = []
      count = 0

      lines=[]
      for i,line in enumerate(data):
        line = re.split(r'[\t,\s]',line)
        lines.append(line)

In [ ]:
lines[:20]

[['*', '0', '17D', '3/3', '0.388993'],
 ['人工', '名詞', '一般', '*', '*', '*', '*', '人工', 'ジンコウ', 'ジンコー'],
 ['知能', '名詞', '一般', '*', '*', '*', '*', '知能', 'チノウ', 'チノー'],
 ['人工', '名詞', '一般', '*', '*', '*', '*', '人工', 'ジンコウ', 'ジンコー'],
 ['知能', '名詞', '一般', '*', '*', '*', '*', '知能', 'チノウ', 'チノー'],
 ['*', '1', '17D', '2/3', '0.613549'],
 ['（', '記号', '括弧開', '*', '*', '*', '*', '（', '（', '（'],
 ['じん', '名詞', '一般', '*', '*', '*', '*', 'じん', 'ジン', 'ジン'],
 ['こうち', '名詞', '一般', '*', '*', '*', '*', 'こうち', 'コウチ', 'コーチ'],
 ['のう', '助詞', '終助詞', '*', '*', '*', '*', 'のう', 'ノウ', 'ノー'],
 ['、', '記号', '読点', '*', '*', '*', '*', '、', '、', '、'],
 ['、', '記号', '読点', '*', '*', '*', '*', '、', '、', '、'],
 ['*', '2', '3D', '0/0', '0.758984'],
 ['AI', '名詞', '一般', '*', '*', '*', '*', '*'],
 ['*', '3', '17D', '1/5', '0.517898'],
 ['〈', '記号', '括弧開', '*', '*', '*', '*', '〈', '〈', '〈'],
 ['エーアイ', '名詞', '固有名詞', '一般', '*', '*', '*', '*'],
 ['〉', '記号', '括弧閉', '*', '*', '*', '*', '〉', '〉', '〉'],
 ['）', '記号', '括弧閉', '*', '*', '*', '*', 

In [ ]:
with open(persed_file) as indata:
        data = indata.read()
        data = data.split('\n')

In [ ]:
test=['* 0 17D 3/3 0.388993',
 '人工\t名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー',
 '知能\t名詞,一般,*,*,*,*,知能,チノウ,チノー',
 '人工\t名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー',
 '知能\t名詞,一般,*,*,*,*,知能,チノウ,チノー',
 '* 1 17D 2/3 0.613549',
 '（\t記号,括弧開,*,*,*,*,（,（,（',
 'じん\t名詞,一般,*,*,*,*,じん,ジン,ジン',
 'こうち\t名詞,一般,*,*,*,*,こうち,コウチ,コーチ',
 'のう\t助詞,終助詞,*,*,*,*,のう,ノウ,ノー',
 '、\t記号,読点,*,*,*,*,、,、,、',
 '、\t記号,読点,*,*,*,*,、,、,、',
 '* 2 3D 0/0 0.758984',
 'AI\t名詞,一般,*,*,*,*,*',
 '* 3 17D 1/5 0.517898',
 '〈\t記号,括弧開,*,*,*,*,〈,〈,〈',
 'エーアイ\t名詞,固有名詞,一般,*,*,*,*',
 '〉\t記号,括弧閉,*,*,*,*,〉,〉,〉',
 '）\t記号,括弧閉,*,*,*,*,）,）,）',
 'と\t助詞,格助詞,引用,*,*,*,と,ト,ト',
 'は\t助詞,係助詞,*,*,*,*,は,ハ,ワ',
 '、\t記号,読点,*,*,*,*,、,、,、',
 '* 4 5D 2/2 1.035972',
 '「\t記号,括弧開,*,*,*,*,「,「,「',
 '『\t記号,括弧開,*,*,*,*,『,『,『',
 '計算\t名詞,サ変接続,*,*,*,*,計算,ケイサン,ケイサン',
 '* 5 9D 0/3 1.243687',
 '（\t記号,括弧開,*,*,*,*,（,（,（',
 '）\t記号,括弧閉,*,*,*,*,）,）,）',
 '』\t記号,括弧閉,*,*,*,*,』,』,』']

In [ ]:
len(test)

30

In [ ]:
test[0][4]

'1'

In [ ]:
a=[]
b=[]
count=0

for i in range(0,30):

  if test[i][0]=="*":
    count+=1
  elif test[i][0]!="*":
    if count ==1:
      b.append(test[i])
      a.append(b)
    elif count == 2:
      
      b=[]
      b.append(test[i])
      a.append(b)
      count=1 



In [ ]:
a[4]

['（\t記号,括弧開,*,*,*,*,（,（,（',
 'じん\t名詞,一般,*,*,*,*,じん,ジン,ジン',
 'こうち\t名詞,一般,*,*,*,*,こうち,コウチ,コーチ',
 'のう\t助詞,終助詞,*,*,*,*,のう,ノウ,ノー',
 '、\t記号,読点,*,*,*,*,、,、,、',
 '、\t記号,読点,*,*,*,*,、,、,、']

## 42.係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．<br>ただし，句読点などの記号は出力しないようにせよ．

- pandasのfor文：https://note.nkmk.me/python-pandas-dataframe-for-iteration/

## 過程

In [18]:
chunks.head()

chunk chunk_id chunk_to
0    人工知能人工知能        0       17
1  （じんこうちのう、、        1       17
2          AI        2        3
3  〈エーアイ〉）とは、        3       17
4        「『計算        4        5

In [13]:
stop_words=[]
for m in morphs:
    if m['pos']=='記号':
        stop_words.append(m['base'])

In [17]:
stop_words=set(stop_words)

In [19]:
id=[]
to=[]
for id in chunks['chunk_id']:
    for to in chunks['chunk_to']:
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27